## torch based DataLoader 가져오기

In [ ]:
import torch
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

from utils.dataset import (
    PCOSDataset,
    create_label_mapping,
    stratified_split_by_pid,
    stratified_pid_kfold,
)
from utils.dataset import HFVisionDataset
from utils.dataset import create_weighted_sampler

from utils.transform import get_transform, SpeckleNoise, AddGaussianNoise
from torchvision import transforms
import pandas as pd
import os 

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
data_root_dir = "/workspace/pcos_dataset/Dataset"
label_path = "/workspace/pcos_dataset/labels/기존_Dataset_info.csv"
result_root_dir = "/workspace/pcos_dataset/results"

label_df = pd.read_csv(label_path)

# 1) transform 생성
train_tf, val_tf = get_transform(
    train_transform=[
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(10),
        transforms.RandomAffine(degrees=0, translate=(0.05,0.05), scale=(0.95,1.05)),
        transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
        # AddGaussianNoise(std=0.02),
        # SpeckleNoise(noise_factor=0.1),
    ]
)

# 2) label mapping
label_mapping = create_label_mapping(label_df, "label")

# 3) Train / Val / Test split (PID 단위 7:1:2)
train_df, val_df, test_df = stratified_split_by_pid(label_df)

# 4) Tune = Train + Val (5-Fold용)
tune_df = pd.concat([train_df, val_df]).reset_index(drop=True)

# 5) PID 기반 5-Fold
folds = stratified_pid_kfold(tune_df, n_splits=5)

In [ ]:
from transformers import (
    AutoModelForImageClassification,
)
model_name = "google/vit-base-patch16-224"

num_labels = len(label_mapping)

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy.compute(references=labels, predictions=preds)["accuracy"],
        "f1": f1.compute(references=labels, predictions=preds, average="macro")["f1"],
    }


In [ ]:
fold_results = []

for fold_idx, (fold_train_df, fold_val_df) in enumerate(folds):
    print(f"\n======== Fold {fold_idx} Training Start ========")

    train_base = PCOSDataset(
        fold_train_df, data_root_dir,
        filename_col="filename",
        label_col="label",
        label_mapping=label_mapping,
        transform=train_tf,
    )

    val_base = PCOSDataset(
        fold_val_df, data_root_dir,
        filename_col="filename",
        label_col="label",
        label_mapping=label_mapping,
        transform=val_tf,
    )

    train_dataset = HFVisionDataset(train_base)
    val_dataset   = HFVisionDataset(val_base)

    # fold마다 새 TrainingArguments / 새 output_dir
    training_args = TrainingArguments(
        output_dir=f"{result_root_dir}/pcos_fold_{fold_idx}",
        learning_rate=5e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=5,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        save_total_limit=2,
        logging_dir=f"{result_root_dir}/logs_fold_{fold_idx}",
    )

    # fold마다 모델을 새로 초기화하는 게 더 정석
    model = AutoModelForImageClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
        ignore_mismatched_sizes=True,
        cache_dir="/workspace/pcos_dataset/models"
    ).to(device)

    model.config.id2label = {int(v): str(k) for k, v in label_mapping.items()}
    model.config.label2id = {str(k): int(v) for k, v in label_mapping.items()}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    metrics = trainer.evaluate()
    print(f"[Fold {fold_idx}] metrics:", metrics)
    fold_results.append(metrics)
